In [1]:
from crowd_nav.configs.config import Config
import numpy as np

for i in range(10):
    #np.random.seed()
    config = Config()
    print(config.env.csl_workspace_type)

simple_corridor
simple_corridor
simple_corridor
simple_corridor
simple_corridor
simple_corridor
simple_corridor
simple_corridor
simple_corridor
simple_corridor


In [10]:
from openai import OpenAI
import base64
import os
from dotenv import load_dotenv

load_dotenv("openai.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

image_paths = ["test2.png"]

# 将图片编码为 base64，用于上传
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def encode_image_from_bytes(image_bytes):
    return base64.b64encode(image_bytes).decode("utf-8")

# 构建图像消息 payload
image_payloads = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{encode_image(path)}",
        }
    }
    for path in image_paths
]
'''
image_payloads = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{encode_image_from_bytes(image_bytes)}"
        }
    }
]
'''

# 提示词
prompt_text = "Suppose you are a wheeled robot performing social navigation tasks in a simulation environment. All green cylinders in the picture are humans. Please judge the current human activity and the robot's current environment based on the image I uploaded. Please select one output from ['walking', 'carrying', 'static', 'talking'] and ['simple_corner', 'simple_corridor'] respectively. Requirement: Only output the judgment content, do not output redundant content, and do not explain why. Output format: human_id, human activity, env, give all the info of detected human in this format"
messages = [
    {"role": "user", "content": [*image_payloads, {"type": "text", "text": prompt_text}]}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",  # 也可尝试 "gpt-4-turbo"
    messages=messages,
    max_tokens=100,
    temperature=0.0,  # 保持回答一致性
)

# 打印回答
print(response.choices[0].message.content)



5, carrying, simple_corridor; 3, static, simple_corridor; 4, static, simple_corridor


In [ ]:
# 分割
entries = [entry.strip() for entry in response.choices[0].message.content.split(";") if entry.strip()]

human_id_list = []
activities = []
scene_type_list = []

for entry in entries:
    parts = entry.split(", ")
    if len(parts) == 3:
        human_id_list.append(parts[0])
        activities.append(parts[1])
        scene_type_list.append(parts[2])
print(human_id_list)
print(scene_type_list)
print(activities)

['5', '3', '4']
['simple_corridor', 'simple_corridor', 'simple_corridor']
['carrying', 'static', 'static']
